In [18]:
# code for web scraping trialing
# name, address, latitude, longitude, country, region, city, contact data
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

In [19]:
URL = "http://trialing-ds.s3-website-eu-west-1.amazonaws.com/"
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")


In [1]:
# print(soup.prettify())

In [21]:
cards = soup.find_all("div", class_ = "card-body shadow p-3")

In [22]:
# name, address, latitude, longitude, country, region, city, contact data
# we assume here that the first "p" is awlays address and if there is a second one
# we use it as phone info. If length is one we set phone NA

def extract_info(cards):
    hospital_id = []
    name = []
    address = []
    city = []
    region = []
    country = []
    phone = []
    long = []
    lat = []
    
    for card in cards:
        hospital_id.append(card.parent.get("id"))
        name.append(card.find("h5").text)
        all_p = card.find_all("p")
        add_info = all_p[0].text
        add_info = re.split(pattern = 'Address: ', string = add_info)[1]
        add_info = re.split(pattern = "\|", string = add_info)
        address.append(add_info[0].strip())
        city.append(add_info[1].strip())
        city_region_info = re.split(pattern = ",", string = add_info[2])
        region.append(city_region_info[0].strip())
        country.append(city_region_info[1].strip())
        if len(all_p) > 1:
            ph_info = all_p[1].text
            ph_info = re.split(pattern = "Phone: ", string = ph_info)[1]
            phone.append(ph_info)
        else:
            phone.append(np.nan)
        link = card.find("a")
        link_url = link["href"]
        lat_info = re.search('\@(-?[\d\.]*)', link_url).group(1)
        long_info = re.search('\@[-?\d\.]*\,([-?\d\.]*)', link_url).group(1)
        lat.append(lat_info)
        long.append(long_info)
        
    df = pd.DataFrame({'hospital_id': hospital_id,
                  'name': name,
                  'addr': address,
                  'city': city,
                  'region': region,
                  'country': country,
                  'phone': phone,
                  'long': long,
                  'lat': lat})
        
    return(df)
    


In [23]:
df = extract_info(cards)

In [24]:
df.head()

,hospital_id,name,addr,city,region,country,phone,long,lat
0,214eb259,Distrito Sanitario de Almería,"Ronda Rector Arolas, S/N, 17600",Almería,Andalucía,Spain,NaN,-3.6727050,40.4331320
1,d7eb4d6b,Hospital Marítimo de Oza,"Avenida Gran Via, 203, 08908",A Coruña,Galicia,Spain,+34 92 510 63 51,2.1214150,41.3970880
2,194f8ead,Hospital do Meixoeiro,"Avenida Ejercito Español, 10, 23007",Vigo,Galicia,Spain,NaN,-16.7146280,28.0499390
3,554b48cd,Centro De Atención Especializada Oroitu,"Calle Eladio Salinero De Los Santos, 6, 06011",Getxo,País Vasco,Spain,+34 97 553 27 59,-5.6984970,43.5454600
4,05022ca2,IIS.ES - Instituto del Sueño en Madrid,"Calle Franz Schubert, 2, 50012",Madrid,Comunidad de Madrid,Spain,+34 96 329 86 59,-0.0360140,39.9972250


In [25]:
df.tail()

,hospital_id,name,addr,city,region,country,phone,long,lat
897,74348948,ICO (Institut Catala D'Oncologia) - Hospital D...,"Avenida De La Feria, S/N, 30510",L' Hospitalet de Llobregat,Cataluña,Spain,+34 98 533 90 19,-3.7067890,40.4313400
898,014c74a5,Hospital Clinic De Barcelona,"Calle Domingo J Manrique, 2, 38006",Barcelona,Cataluña,Spain,NaN,-8.6502210,42.4300060
899,98364377,Hospital Universitari Vall D'Hebron,"Calle Doctor Repolles, 2, 44600",Barcelona,Cataluña,Spain,+34 98 831 52 95,-1.6237700,42.8161890
900,a9993451,Hospital Del Mar,"Calle Mister Blisse, S/N, 35212",Barcelona,Cataluña,Spain,+34 99 371 13 97,-0.3302310,39.4766240
901,46253146,Hospital De La Santa Creu i Sant Pau,"Avenida Gil De Atrocillo, S/N, 12500",Barcelona,Cataluña,Spain,NaN,-8.6842200,42.2150000


In [27]:
df.to_csv('../../data/hospital_web.csv', index=False, encoding='utf-8')